<h1><center>NDA Client GTM India</center></h1>
<center>This workbook conducts data collection, cleaning, formatting, and upload for GTM analysis of Indian B2C market segment.<br>
_______________________________________________________________________________________________________________</center>

## Housekeeping
We are going to use 2 main types of cells in Jupyter Notebook: <strong>Code</strong> and <strong>Markdown</strong> cells.
<br><br>This cell is a markdown cell and can be used to describe methodology of the analysis outputs. Markdown cells are formatted using basic HTML.
<br><br>For example, to make a heading you can use \<h1>, \</h1> or # foolowed by the space. Double click on this cell to see the syntax inside. 
<br><br>With the same logic you can use any <strong>tags</strong>. Magority of tags exists in pairs - opening and closing tag (just like the heading one). So if, for example, you want to make your text look bold you need you use \<strong> tag. The final code will look like this \<strong><strong>Your bold text</strong>\</strong>.<br><br>Another important starter tag is \<br> and it does not need a closing pair. \<br> will act as an Enter and put the text after it to the next line.<br>

In the Code cells any text startting with \# is <strong>comments</strong>. They are not considered a part of the code by Python and can be used to explain the purpose of the following code.If you want to put a multuline comment (or skip some piece of code without deleting it) surround it by tripple quotes.Result will look similar to this:<br>
<br>
<center>'''Your<br>
multiline<br>
comment'''</center>
<br>
Some steps of the analysis should be done only ones and once completed should be commented out. For big chunks of code it can be handy to use multiline comment scrips (enclose your code in triple quotes) instead of commenting out every line.
<center>_______________________________________________________________________________________________________________</center>

## Establishing connection with MySQL
The next block of code sells are importing packages and extablishing connection with MySQL.
We are using AWS RDS to run our database on MySQL engine. Please note the credentials for extablishing the connection to the database.<br>
- host =*Sensored*<br>
- port = *Sensored*<br>
- user = *Sensored*<br>
- password = *Sensored*<br>
- db = *Sensored*<br>

Database schema can be edited in MySQL Workbench and through <strong>MySQL Python Connect (pymysql)</strong> and <strong>sqlalchemy</strong> packages. In this Notebook we are going to use sqlalchemy to write data to the AWS and MySQL Python Connect for data retrieving.<br><br>

In [ ]:
# general import statement

import pandas as pd
import numpy as np
import geopandas as gpd
import os
import fiona
import pymysql as sql
import openpyxl
from sqlalchemy import create_engine

In [ ]:
#establishing a connection with MySQL Connector

conn = sql.connect(host = #enter host name,
                   port = #enter port number,
                   user = #enter user,
                   password = #enter password,
                   db = #enter database name)

In [ ]:
#setting up a cursor (you "Python-powered hand" in MySQL)

cursor=conn.cursor()

In [ ]:
#progress bars function definition

def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
#working directory check (and some common commands)

#os.getcwd()
#os.chdir('..')

<br><br>


## Working with Maps
In this section of the analysis we import and study maps to be used in the analysis. For everycountry of analysis at least 2 levels of granularity will be needed. We will use several binaryvariables and agregate them to continuous values on the higher level.<br>

In case of India, we will be working with 2nd and 3rd administrative level maps. Results of the analysis will be aggregated and presented on the 2nd administrative level (Distcricts), but early section of the analysis will be based on 3rd (Subdistricts).

- Level 0: India
- Level 1: State
- Level 2: Distctricts
- Level 3: Subdistcricts

In [ ]:
#importing all the Statistical areas boundaries

India_shp = gpd.read_file('ind_adm_shp/IND_adm4v4.shp')

India_shp = India_shp.drop(['Subdt_LGD', 'Dist_LGD', 'State_LGD', 'JSID', 'layer', 'path'], axis = 1)

India_shp['SD_CODE'] = India_shp['stcode11'] + India_shp['dtcode11'] + India_shp['sdtcode11']
India_shp['D_CODE'] = India_shp['stcode11'] + India_shp['dtcode11']

In [ ]:
#importing mobile circles
M_circles = gpd.read_file('ind_adm_shp/Mobile circles.shp')

M_circles = M_circles.drop(['Subdt_LGD', 'Dist_LGD', 'State_LGD', 'JSID', 'layer'], axis = 1)

M_circles['SD_CODE'] = M_circles['stcode11'] + M_circles['dtcode11'] + M_circles['sdtcode11']
M_circles['D_CODE'] = M_circles['stcode11'] + M_circles['dtcode11']

# Retail ISP (B2C)

## Part 1:  Affordability
Next block of code uses previous analysis of RWI distribution to constract affordability estimates for various scenarios. India has a significant difference between urban and rural income distribution and we will assume thta RWI is a mixture of two. In this case we use <strong>burr</strong> distribution for urban RWI distribution <strong>kstwobign</strong> for rural RWI distribution.<br><br>
<strong>Burr</strong> has 4 parameters with following estimated values:<br> 
<strong>c:</strong> 24.1<br>
<strong>d:</strong> 0.27<br>
<strong>loc:</strong> -0.44<br>
<strong>scale:</strong> 1.81<br>
<br>
<strong>Kstwobign</strong> has 2 parameters with following estimated values:<br> 
<strong>loc:</strong> -1.41<br>
<strong>scale:</strong> 1.85<br>



In [ ]:
#importing resulting distributions

from scipy.stats import burr, kstwobign

In [ ]:
#rural/urban split tables

R_U = pd.read_excel('Data/RURAL_URBAN.xlsx', converters={'CODE': str, 'D_CODE': str})

In [ ]:
#employment tables

EMPL = pd.read_excel('Data/EMPL.xlsx', converters={'D_CODE': str})
EMPL = EMPL[EMPL['Empl'] != 'Total']
EMPL['Empl'] = EMPL['Empl'].replace('None', 0).astype(int)


EMPL_U = EMPL[EMPL['T_R_U'] == 'Urban']
EMPL_R = EMPL[EMPL['T_R_U'] == 'Rural']

In [ ]:
#calculating weigted average number of employed per household

EMPL_U['HH_PC'] = EMPL_U['HH_T'] / EMPL_U.groupby('D_CODE')['HH_T'].transform('sum')
EMPL_U['N_EMPL'] = EMPL_U['HH_PC']*EMPL_U['Empl']

EMPL_R['HH_PC'] = EMPL_R['HH_T'] / EMPL_R.groupby('D_CODE')['HH_T'].transform('sum')
EMPL_R['N_EMPL'] = EMPL_R['HH_PC']*EMPL_R['Empl']

In [ ]:
#grouping the results

EMPL_U_G = EMPL_U.groupby('D_CODE', as_index = False).agg({'N_EMPL': 'sum'})
EMPL_R_G = EMPL_R.groupby('D_CODE', as_index = False).agg({'N_EMPL': 'sum'})

In [ ]:
#replacing missing data

EMPL_U_G['N_EMPL'] = EMPL_U_G['N_EMPL'].replace(0,1.5)
EMPL_R_G['N_EMPL'] = EMPL_R_G['N_EMPL'].replace(0,1.5)

In [ ]:
#merging with rural/urban split data (one to many merge)

R_U_M = pd.merge(R_U, EMPL_R_G, on = 'D_CODE', how = 'outer')
R_U_M = pd.merge(R_U_M, EMPL_U_G, on = 'D_CODE', how = 'outer', suffixes=('_R', '_U'))

In [ ]:
#setting parameters
GNI = 1920
tax = 0.18
price = [5,10,15,20,25]
price = list(map(lambda x: x *(1+tax), price))
sow = [0.03,0.04,0.05,0.06]

HH_income = pd.DataFrame(columns = ['Scenario', 'HH_income'])
for pr in price:
    for s in sow:
        x = (pr*12)/s

        Scenario = '$' + str(pr/(1+tax))+'_'+str(s)
        Income = x
        d = {'Scenario': Scenario, 'HH_income': Income}
        df = pd.DataFrame(data=d, index=[0])
        HH_income = HH_income.append(df)
        
HH_income = HH_income.reset_index().drop(['index'],axis=1)

In [ ]:
#calculation RWI
columns = []
for index, row in HH_income.iterrows():
    col_name_U = 'RWI_U_'+ row['Scenario']
    col_name_R = 'RWI_R_'+ row['Scenario']
    R_U_M[col_name_U] = ((row['HH_income']/R_U_M['N_EMPL_U'])/GNI)-1
    R_U_M[col_name_R] = ((row['HH_income']/R_U_M['N_EMPL_R'])/GNI)-1
    columns.append([col_name_U,col_name_R])

In [ ]:
#calculating affordability
for index, col in R_U_M.iteritems():    
    if index[:5] == 'RWI_U' or index[:5] == 'RWI_R':
        col_name = 'Afford_HH_' + index[6:]
        R_U_M[col_name] = 1-(R_U_M['Urban_PC']*burr.cdf(R_U_M[index], c=24.1, d=0.27, loc=-0.44, scale=1.81)+R_U_M['Rural_PC']*kstwobign.cdf(R_U_M[index], loc=-1.41, scale=1.85))    
    else: 
        continue       

In [ ]:
# calculating deciles

R_U_M['HH_INC_Q10'] = R_U_M['Urban_PC']*(burr.ppf(0.1, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.1, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q20'] = R_U_M['Urban_PC']*(burr.ppf(0.2, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.2, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q30'] = R_U_M['Urban_PC']*(burr.ppf(0.3, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.3, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q40'] = R_U_M['Urban_PC']*(burr.ppf(0.4, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.4, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q50'] = R_U_M['Urban_PC']*(burr.ppf(0.5, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.5, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q60'] = R_U_M['Urban_PC']*(burr.ppf(0.6, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.6, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q70'] = R_U_M['Urban_PC']*(burr.ppf(0.7, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.7, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q80'] = R_U_M['Urban_PC']*(burr.ppf(0.8, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.8, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q90'] = R_U_M['Urban_PC']*(burr.ppf(0.9, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.9, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']
R_U_M['HH_INC_Q100'] = R_U_M['Urban_PC']*(burr.ppf(0.999, c=24.1, d=0.27, loc=-0.44, scale=1.81)+1)*GNI*R_U_M['N_EMPL_U']+R_U_M['Rural_PC']*(kstwobign.ppf(0.999, loc=-1.41, scale=1.85)+1)*GNI*R_U_M['N_EMPL_R']

In [ ]:
#rolling up to district level

R_U_M['HH_PC'] = R_U_M['Total'] / R_U_M.groupby('D_CODE')['Total'].transform('sum')

for index, col in R_U_M.iteritems():    
    if index[:6] == 'Afford':
        col_name = index + '_PC'
        R_U_M[col_name] = R_U_M['HH_PC']*R_U_M[index]
    else: 
        continue 
        
R_U_M['HH_INC_Q10_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q10']
R_U_M['HH_INC_Q20_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q20']
R_U_M['HH_INC_Q30_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q30']
R_U_M['HH_INC_Q40_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q40']
R_U_M['HH_INC_Q50_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q50']
R_U_M['HH_INC_Q60_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q60']
R_U_M['HH_INC_Q70_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q70']
R_U_M['HH_INC_Q80_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q80']
R_U_M['HH_INC_Q90_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q90']
R_U_M['HH_INC_Q100_PC'] = R_U_M['HH_PC']*R_U_M['HH_INC_Q100']


In [ ]:
#filtering by correct columns
x = [1]
x.extend(list(range(80,111)))
R_U_G = R_U_M.iloc[:,x]

afford = R_U_M.iloc[:,81:101].columns

R_U_G = R_U_G.groupby('D_CODE', as_index = False).sum()

afford = list(afford)

In [ ]:
#sensitivity analysis for service price
'''import matplotlib.pyplot as plt
import seaborn as sns

GNI = 1920

SOW_low = 0.04
empl_low = 1

SOW_med = 0.06
empl_med = 1.2

SOW_high = 0.08
empl_high = 1.8

x_low = []
y_low = []

x_med = []
y_med = []

x_high = []
y_high = []

for i in range(101):
    price = i
    income = (price*12)/SOW_low/empl_low
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_low.append(i)
    y_low.append(1-prob)
    
for i in range(101):
    price = i
    income = (price*12)/SOW_med/empl_med
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_med.append(i)
    y_med.append(1-prob)
        
for i in range(101):
    price = i
    income = (price*12)/SOW_high/empl_high
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_high.append(i)
    y_high.append(1-prob)
    
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_theme(style="darkgrid")
sns.lineplot(x=x_low, y=y_low).set(title='Price sensitivity', xlabel = "Service Price", ylabel = "Affordability")
sns.lineplot(x=x_med, y=y_med)
sns.lineplot(x=x_high, y=y_high)
plt.legend(labels=["4% share of wallet, 1 employed","6% share of wallet, 1.2 employed", "8% share of wallet, 1.8 employed"])'''

In [ ]:
#sensitivity analysis for share of wallet
'''
GNI = 1920

price_low = 10
empl_low = 1

price_med = 15
empl_med = 1.2

price_high = 25
empl_high = 1.8

x_low = []
y_low = []

x_med = []
y_med = []

x_high = []
y_high = []

for i in np.arange(0, 0.5, 0.01):
    SOW = i
    income = (price_low*12)/SOW/empl_low
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_low.append(i)
    y_low.append(1-prob)
    
for i in np.arange(0, 0.5, 0.01):
    SOW = i
    income = (price_med*12)/SOW/empl_med
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_med.append(i)
    y_med.append(1-prob)
    
for i in np.arange(0, 0.5, 0.01):
    SOW = i
    income = (price_high*12)/SOW/empl_high
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_high.append(i)
    y_high.append(1-prob)
    
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_theme(style="darkgrid")
sns.lineplot(x=x_low, y=y_low).set(title='Share of wallet sensitivity', xlabel = "Share of wallet", ylabel = "Affordability")
sns.lineplot(x=x_med, y=y_med)
sns.lineplot(x=x_high, y=y_high)
plt.legend(labels=["$10 service price, 1 employed","$15 service price, 1.2 employed", "$25 service price, 1.8 employed"])'''

In [ ]:
#sensitivity analysis for number of employed people per household
'''
GNI = 1920

price_low = 10
SOW_low = 0.04

price_med = 15
SOW_med = 0.06

price_high = 25
SOW_high = 0.08

x_low = []
y_low = []

x_med = []
y_med = []

x_high = []
y_high = []

for i in np.arange(1, 3, 0.1):
    empl = i
    income = (price_low*12)/SOW_low/empl
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_low.append(i)
    y_low.append(1-prob)
    
for i in np.arange(1, 3, 0.1):
    empl = i
    income = (price_med*12)/SOW_med/empl
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_med.append(i)
    y_med.append(1-prob)
    
for i in np.arange(1, 3, 0.1):
    empl = i
    income = (price_high*12)/SOW_high/empl
    RWI = income/GNI-1
    prob = 0.8*burr.cdf(RWI, c=24.1, d=0.27, loc=-0.44, scale=1.81)+0.2*kstwobign.cdf(RWI, loc=-1.41, scale=1.85)
    x_high.append(i)
    y_high.append(1-prob)
    
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_theme(style="darkgrid")
sns.lineplot(x=x_low, y=y_low).set(title='Share of wallet sensitivity', xlabel = "# of employed", ylabel = "Affordability")
sns.lineplot(x=x_med, y=y_med)
sns.lineplot(x=x_high, y=y_high)
plt.legend(labels=["$10 service price, 4% share of wallet","$15 service price, 6% share of wallet", "$25 service price, 8% share of wallet"])'''

## Part 2: Fixed and mobile speed tests
<br>
First part of the analysis is to construct a table containing Socio-Demographic and Speed test data. Follow the code below to see the analysis steps.

In [ ]:
# Uploading Mlab data (Subdistricts x Days of speed tests) (3 parts)

Mlab_IND_Subdistrics_B2C = pd.read_csv('Data/Mlab_IND_B2C_p1.csv',converters={'SD_CODE': str}).append(pd.read_csv('Data/Mlab_IND_B2C_p2.csv',converters={'SD_CODE': str})).append(pd.read_csv('Data/Mlab_IND_B2C_p3.csv',converters={'SD_CODE': str}))


In [ ]:
#adding lost leading zero

max_length_ID_1 = Mlab_IND_Subdistrics_B2C['SD_CODE'].map(len).max()
Mlab_IND_Subdistrics_B2C['SD_CODE'] = Mlab_IND_Subdistrics_B2C['SD_CODE'].apply(lambda x: '0'*(max_length_ID_1 - len(x)) + x)

In [ ]:
# Grouping Mlab data to arrive at aggregated measured of speed test attributes by Subdistrict

Mlab_IND_Subdistrics_B2C_G = Mlab_IND_Subdistrics_B2C.groupby('SD_CODE', as_index = False).agg({'Mbps':  'median',
                                                                   'download_MED':  'median',
                                                                   'maxMbps':  'max',
                                                                   'minMbps':  'min',
                                                                   'number_speed_tests':  'sum',
                                                                   'latency' : 'mean'})

In [ ]:
# Creating speed test "buckets"

Mlab_IND_Subdistrics_B2C_G['2Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 2).astype(int)
Mlab_IND_Subdistrics_B2C_G['5Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 5).astype(int)
Mlab_IND_Subdistrics_B2C_G['10Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 10).astype(int)
Mlab_IND_Subdistrics_B2C_G['15Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 15).astype(int)
Mlab_IND_Subdistrics_B2C_G['20Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 20).astype(int)
Mlab_IND_Subdistrics_B2C_G['25Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 25).astype(int)
Mlab_IND_Subdistrics_B2C_G['50Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 50).astype(int)
Mlab_IND_Subdistrics_B2C_G['100Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 100).astype(int)
Mlab_IND_Subdistrics_B2C_G['400Mbps'] = (Mlab_IND_Subdistrics_B2C_G['download_MED'] > 400).astype(int)

In [ ]:
#importing 4G coverage data (not available)

Vi_LTE = gpd.read_file('Data/Ookla/Results/Vi_LTE.shp')
MTNL_LTE = gpd.read_file('Data/Ookla/Results/MTNL_LTE.shp')
BSNL_LTE = gpd.read_file('Data/Ookla/Results/BSNL_LTE.shp')
Airtel_LTE = gpd.read_file('Data/Ookla/Results/Airtel_LTE.shp')

In [ ]:
#data cleaning and column renaming

Vi_LTE = Vi_LTE.drop(['Subdt_LGD', 'Dist_LGD', 'State_LGD', 'JSID', 'layer', 'path', 'geometry'], axis = 1).rename(columns={'area_Vi__1': 'area_Vi_PC'})
MTNL_LTE = MTNL_LTE.drop(['Subdt_LGD', 'Dist_LGD', 'State_LGD', 'JSID', 'layer', 'path', 'geometry'], axis = 1).rename(columns={'area_MTNL_': 'area_MTNL_PC'})
BSNL_LTE = BSNL_LTE.drop(['Subdt_LGD', 'Dist_LGD', 'State_LGD', 'JSID', 'layer', 'path', 'geometry'], axis = 1).rename(columns={'area_BSNL_': 'area_BSNL_PC'})
Airtel_LTE = Airtel_LTE.drop(['Subdt_LGD', 'Dist_LGD', 'State_LGD', 'JSID', 'layer', 'path', 'geometry'], axis = 1).rename(columns={'area_Air_1': 'area_Airtel_PC'})

In [ ]:
#merging all mobile operators
mobile_coverage = pd.merge(Vi_LTE, MTNL_LTE[['SD_CODE', 'area_MTNL_PC']], on = 'SD_CODE', how = 'outer')
mobile_coverage = pd.merge(mobile_coverage, BSNL_LTE[['SD_CODE', 'area_BSNL_PC']], on = 'SD_CODE', how = 'outer')
mobile_coverage = pd.merge(mobile_coverage, Airtel_LTE[['SD_CODE', 'area_Airtel_PC']], on = 'SD_CODE', how = 'outer')

In [ ]:
#Importing of demographic data and standartization of unique state/district/subdistrict codes

DEM_STAT_IND_Subdistrics_B2C = pd.read_excel('Data/DEM_STAT_IND.xlsx')

DEM_STAT_IND_Subdistrics_B2C['State_Code'] = DEM_STAT_IND_Subdistrics_B2C['State_Code'].astype(str)
DEM_STAT_IND_Subdistrics_B2C['District_Code'] = DEM_STAT_IND_Subdistrics_B2C['District_Code'].astype(str)
DEM_STAT_IND_Subdistrics_B2C['SubDistrict_Code'] = DEM_STAT_IND_Subdistrics_B2C['SubDistrict_Code'].astype(str)

max_length_ID_1 = DEM_STAT_IND_Subdistrics_B2C['State_Code'].map(len).max()
DEM_STAT_IND_Subdistrics_B2C['State_Code'] = DEM_STAT_IND_Subdistrics_B2C['State_Code'].apply(lambda x: '0'*(max_length_ID_1 - len(x)) + x)

max_length_ID_2 = DEM_STAT_IND_Subdistrics_B2C['District_Code'].map(len).max()
DEM_STAT_IND_Subdistrics_B2C['District_Code'] = DEM_STAT_IND_Subdistrics_B2C['District_Code'].apply(lambda x: '0'*(max_length_ID_2 - len(x)) + x)

max_length_ID_3 = DEM_STAT_IND_Subdistrics_B2C['SubDistrict_Code'].map(len).max()
DEM_STAT_IND_Subdistrics_B2C['SubDistrict_Code'] = DEM_STAT_IND_Subdistrics_B2C['SubDistrict_Code'].apply(lambda x: '0'*(max_length_ID_3 - len(x)) + x)

DEM_STAT_IND_Subdistrics_B2C['SD_CODE'] = DEM_STAT_IND_Subdistrics_B2C['State_Code']+DEM_STAT_IND_Subdistrics_B2C['District_Code']+DEM_STAT_IND_Subdistrics_B2C['SubDistrict_Code']
DEM_STAT_IND_Subdistrics_B2C['D_CODE'] = DEM_STAT_IND_Subdistrics_B2C['State_Code']+DEM_STAT_IND_Subdistrics_B2C['District_Code']

In [ ]:
#merging Mlab and Demographic data by Subdistrict code

IND_Subdistrics_B2C = pd.merge(Mlab_IND_Subdistrics_B2C_G, DEM_STAT_IND_Subdistrics_B2C, on = 'SD_CODE', how = 'outer')
IND_Subdistrics_B2C = pd.merge(IND_Subdistrics_B2C, mobile_coverage, on = 'SD_CODE', how = 'outer')

In [ ]:
#weighting speed test metrics by number of households inside each subdistrict in relation to district

IND_Subdistrics_B2C['HH_PC'] = IND_Subdistrics_B2C['HH_T'] / IND_Subdistrics_B2C.groupby('D_CODE')['HH_T'].transform('sum')

IND_Subdistrics_B2C['AVG_MBPS'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['Mbps']
IND_Subdistrics_B2C['MED_MBPS'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['download_MED']
IND_Subdistrics_B2C['MAX_MED_MBPS'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['maxMbps']

IND_Subdistrics_B2C['2Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['2Mbps']
IND_Subdistrics_B2C['5Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['5Mbps']
IND_Subdistrics_B2C['10Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['10Mbps']
IND_Subdistrics_B2C['15Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['15Mbps']
IND_Subdistrics_B2C['20Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['20Mbps']
IND_Subdistrics_B2C['25Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['25Mbps']
IND_Subdistrics_B2C['50Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['50Mbps']
IND_Subdistrics_B2C['100Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['100Mbps']
IND_Subdistrics_B2C['400Mbps_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['400Mbps']

IND_Subdistrics_B2C['area_Vi_cover_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['area_Vi_PC']
IND_Subdistrics_B2C['area_MTNL_cover_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['area_MTNL_PC']
IND_Subdistrics_B2C['area_BSNL_cover_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['area_BSNL_PC']
IND_Subdistrics_B2C['area_Airtel_cover_PC'] = IND_Subdistrics_B2C['HH_PC']*IND_Subdistrics_B2C['area_Airtel_PC']

In [ ]:
#groupling by district

IND_Subdistrics_B2C_G = IND_Subdistrics_B2C.groupby('D_CODE', as_index = False).agg({'AVG_MBPS': 'sum',
                                                                               'MED_MBPS' : 'sum',
                                                                               'MAX_MED_MBPS' : 'sum',
                                                                               'number_speed_tests': 'sum',
                                                                               'latency' : 'mean',
                                                                               '2Mbps_PC': 'sum',
                                                                               '5Mbps_PC': 'sum',
                                                                               '10Mbps_PC': 'sum',
                                                                               '15Mbps_PC': 'sum',
                                                                               '20Mbps_PC': 'sum',
                                                                               '25Mbps_PC': 'sum',
                                                                               '50Mbps_PC': 'sum',
                                                                               '100Mbps_PC': 'sum',
                                                                               '400Mbps_PC': 'sum',
                                                                               'Population': 'sum',
                                                                               'Inhabited Villages' : 'sum',
                                                                               'Uninhabited Villages' : 'sum',
                                                                               'Number of towns' : 'sum',
                                                                               'HH_T' : 'sum',
                                                                               'Area' : 'sum',
                                                                               'PD' : 'mean',
                                                                               'area_Vi_cover_PC': 'sum',
                                                                               'area_MTNL_cover_PC': 'sum',
                                                                               'area_BSNL_cover_PC': 'sum',
                                                                               'area_Airtel_cover_PC': 'sum'})

In [ ]:
#uploading to the database

#IND_Subdistrics_B2C_G.to_sql(name='IND_B2C', con=engine, if_exists = 'replace', chunksize = 10000, index=False)

## Part 3: Additional Data

### MDUs

In [ ]:
#importing & preperation of MDU data

MDU = pd.read_excel('Data/MDU Data.xlsx', sheet_name = "Ownership", converters = ({'ST_CODE': str, 'D_CODE': str}))

max_length_ID_1 = MDU['ST_CODE'].map(len).max()
MDU['ST_CODE'] = MDU['ST_CODE'].apply(lambda x: '0'*(max_length_ID_1 - len(x)) + x)

max_length_ID_2 = MDU['D_CODE'].map(len).max()
MDU['D_CODE'] = MDU['D_CODE'].apply(lambda x: '0'*(max_length_ID_2 - len(x)) + x)

MDU['CODE'] = MDU['ST_CODE'] + MDU['D_CODE']

In [ ]:
#filtering and dropping columns

MDU_F = MDU[(MDU['T_R_U'] == 'Urban')&(MDU['OWN_STATUS'] == 'Rented')&(MDU['HH_SIZE'].isin(['1','2','3','4','5','6-8']))].drop(['4_ROOM', '5_ROOM', '6_ROOM'], axis = 1)
HH_T = MDU[(MDU['T_R_U'] == 'Total')&(MDU['OWN_STATUS'] == 'Total')&(MDU['HH_SIZE'] == 'All Households')].drop(['STUDIO','1_ROOM','2_ROOM','3_ROOM','4_ROOM', '5_ROOM', '6_ROOM'], axis = 1)

In [ ]:
#calculating total number of MDUs

MDU_F['MDU_T'] = MDU_F['STUDIO']+MDU_F['1_ROOM']+MDU_F['2_ROOM']+MDU_F['3_ROOM']

In [ ]:
#aggregating to higher level

MDU_F_G = MDU_F.groupby('CODE', as_index = False).agg({'MDU_T': 'sum'})

In [ ]:
#calculating total number of households

HH_T_G = HH_T.groupby('CODE', as_index = False).agg({'HH_T': 'sum'})

In [ ]:
#merging MDU with HH_T

MDU_T = pd.merge(MDU_F_G, HH_T_G, on ='CODE', how = 'inner')

In [ ]:
#calculating # of MDU per area

MDU_T['MDU_PC'] = MDU_T['MDU_T']/MDU_T['HH_T']

In [ ]:
#getting rid of HH_T

MDU_T = MDU_T.drop(['HH_T'], axis = 1)

### HH expenditure

In [ ]:
#importing all expenditure records
exp = pd.read_csv('data\Expenditure.txt', delimiter = "\t", names=['v1_HHID',\
                                                                   'v2',\
                                                                   'v3',\
                                                                   'v4',\
                                                                   'v5',\
                                                                   'v6',\
                                                                   'v7_R_U',\
                                                                   'v8_ST_CODE',\
                                                                   'v9_ST_NAME',\
                                                                   'v10_D_NAME',\
                                                                   'v11',\
                                                                   'v12',\
                                                                   'v13',\
                                                                   'v14',\
                                                                   'v15',\
                                                                   'v16',\
                                                                   'v17',\
                                                                   'v18',\
                                                                   'v19',\
                                                                   'v20_ITEM_CODE',\
                                                                   'v21_SPEND_RSS',\
                                                                   'v22',\
                                                                   'v23',\
                                                                   'v24',\
                                                                   'v25',\
                                                                   'v26'], converters = ({'v8_ST_CODE': str}))

exp_food = pd.read_csv('data\Expenditure_MILK.txt', delimiter = "\t", names=['v1',\
                                                                   'v2',\
                                                                   'v3',\
                                                                   'v4',\
                                                                   'v5',\
                                                                   'v6_R_U',\
                                                                   'v7_ST_CODE',\
                                                                   'v8',\
                                                                   'v9',\
                                                                   'v10',\
                                                                   'v11',\
                                                                   'v12',\
                                                                   'v13',\
                                                                   'v14',\
                                                                   'v15',\
                                                                   'v16',\
                                                                   'v17',\
                                                                   'v18',\
                                                                   'v19_ITEM_CODE',\
                                                                   'v20',\
                                                                   'v21',\
                                                                   'v22',\
                                                                   'v23_SPEND_RSS',\
                                                                   'v24',\
                                                                   'v25',\
                                                                   'v26',\
                                                                   'v27',\
                                                                   'v28_HHID',\
                                                                   'v29_ST_NAME',\
                                                                   'v30_D_NAME',\
                                                                   'v31',\
                                                                   'v32'], converters = ({'v7_ST_CODE': str}))

exp_misc = pd.read_csv('data\Expenditure_misc.txt', delimiter = "\t", names=['v1',\
                                                                   'v2',\
                                                                   'v3',\
                                                                   'v4',\
                                                                   'v5',\
                                                                   'v6_R_U',\
                                                                   'v7_ST_CODE',\
                                                                   'v8',\
                                                                   'v9',\
                                                                   'v10',\
                                                                   'v11',\
                                                                   'v12',\
                                                                   'v13',\
                                                                   'v14',\
                                                                   'v15',\
                                                                   'v16',\
                                                                   'v17',\
                                                                   'v18',\
                                                                   'v19_ITEM_CODE',\
                                                                   'v20_SPEND_RSS',\
                                                                   'v21',\
                                                                   'v22',\
                                                                   'v23',\
                                                                   'v24_HHID',\
                                                                   'v25_ST_NAME',\
                                                                   'v26_D_NAME',\
                                                                   'v27',\
                                                                   'v28'], converters = ({'v7_ST_CODE': str}))

exp_summary = pd.read_csv('data\Expenditure_summary.txt', delimiter = "\t", names=['v1',\
                                                                   'v2',\
                                                                   'v3',\
                                                                   'v4',\
                                                                   'v5',\
                                                                   'v6_R_U',\
                                                                   'v7_ST_CODE',\
                                                                   'v8',\
                                                                   'v9',\
                                                                   'v10',\
                                                                   'v11',\
                                                                   'v12',\
                                                                   'v13',\
                                                                   'v14',\
                                                                   'v15',\
                                                                   'v16',\
                                                                   'v17',\
                                                                   'v18',\
                                                                   'v19_ITEM_CODE',\
                                                                   'v20_SPEND_RSS',\
                                                                   'v21',\
                                                                   'v22',\
                                                                   'v23',\
                                                                   'v24_HHID',\
                                                                   'v25_ST_NAME',\
                                                                   'v26_D_NAME',\
                                                                   'v27',\
                                                                   'v28'], converters = ({'v7_ST_CODE': str}))

In [ ]:
#dropping useless columns
exp = exp.drop(['v2','v3', 'v4', 'v5', 'v6', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v22', 'v23', 'v24', 'v25', 'v26'], axis = 1)
exp_food = exp_food.drop(['v1', 'v2','v3', 'v4', 'v5', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v20', 'v21', 'v22', 'v24', 'v25', 'v26', 'v27', 'v31', 'v32'], axis = 1)
exp_misc = exp_misc.drop(['v1', 'v2','v3', 'v4', 'v5', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v21', 'v22', 'v23', 'v27', 'v28'], axis = 1)
exp_summary = exp_summary.drop(['v1', 'v2','v3', 'v4', 'v5', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v21', 'v22', 'v23', 'v27', 'v28'], axis = 1)

In [ ]:
#expenditure on services 2015
exp_R = exp[exp['v7_R_U']==1]
exp_U = exp[exp['v7_R_U']==2]

exp_R['v8_ST_CODE'] = exp_R['v8_ST_CODE'].str[:-1]
exp_U['v8_ST_CODE'] = exp_U['v8_ST_CODE'].str[:-1]

#expenditure on food 2011
exp_food = exp_food[exp_food['v19_ITEM_CODE'].isin([129,159,169,179,189,199,219,239,249,269,279,289,299,309])]
exp_food_R = exp_food[exp_food['v6_R_U']==1]
exp_food_U = exp_food[exp_food['v6_R_U']==2]

exp_food_R['v7_ST_CODE'] = exp_food_R['v7_ST_CODE'].str[:-1]
exp_food_U['v7_ST_CODE'] = exp_food_U['v7_ST_CODE'].str[:-1]

#expenditure on misc 2011
exp_misc_R = exp_misc[exp_misc['v6_R_U']==1]
exp_misc_U = exp_misc[exp_misc['v6_R_U']==2]

exp_misc_R['v7_ST_CODE'] = exp_misc_R['v7_ST_CODE'].str[:-1]
exp_misc_U['v7_ST_CODE'] = exp_misc_U['v7_ST_CODE'].str[:-1]

exp_housing_R = exp_misc_R[exp_misc_R['v19_ITEM_CODE'].isin([520,522,523,539,540])]
exp_taxes_R = exp_misc_R[exp_misc_R['v19_ITEM_CODE'].isin([541])]
exp_extra_R = exp_misc_R[exp_misc_R['v19_ITEM_CODE'].isin([439,449,479,499])]
exp_mobile_R = exp_misc_R[exp_misc_R['v19_ITEM_CODE'].isin([488])]
exp_landline_R = exp_misc_R[exp_misc_R['v19_ITEM_CODE'].isin([487])]
exp_internet_R = exp_misc_R[exp_misc_R['v19_ITEM_CODE'].isin([496])]

exp_housing_U = exp_misc_U[exp_misc_U['v19_ITEM_CODE'].isin([520,522,523,539,540])]
exp_taxes_U = exp_misc_U[exp_misc_U['v19_ITEM_CODE'].isin([541])]
exp_extra_U = exp_misc_U[exp_misc_U['v19_ITEM_CODE'].isin([439,449,479,499])]
exp_mobile_U = exp_misc_U[exp_misc_U['v19_ITEM_CODE'].isin([488])]
exp_landline_U = exp_misc_U[exp_misc_U['v19_ITEM_CODE'].isin([487])]
exp_internet_U = exp_misc_U[exp_misc_U['v19_ITEM_CODE'].isin([496])]

#expenditure summary 2011
exp_summary_R = exp_summary[exp_summary['v6_R_U']==1]
exp_summary_U = exp_summary[exp_summary['v6_R_U']==2]

exp_summary_R['v7_ST_CODE'] = exp_summary_R['v7_ST_CODE'].str[:-1]
exp_summary_U['v7_ST_CODE'] = exp_summary_U['v7_ST_CODE'].str[:-1]

In [ ]:
#preparing TOTAL EXPENDITURE df
exp_R_total = exp_summary_R.groupby(['v7_ST_CODE', 'v24_HHID'], as_index = False).agg({'v20_SPEND_RSS': 'sum'})
exp_U_total = exp_summary_U.groupby(['v7_ST_CODE', 'v24_HHID'], as_index = False).agg({'v20_SPEND_RSS': 'sum'})

exp_R_total = exp_R_total.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})
exp_U_total = exp_U_total.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})

exp_R_total['HH_R_EXP_T'] = exp_R_total['v20_SPEND_RSS']*0.013
exp_U_total['HH_U_EXP_T'] = exp_U_total['v20_SPEND_RSS']*0.013

exp_R_total = exp_R_total.drop(['v20_SPEND_RSS'], axis = 1)
exp_U_total = exp_U_total.drop(['v20_SPEND_RSS'], axis = 1)

In [ ]:
#preparing FOOD EXPENDITURE df
exp_food_R = exp_food_R.groupby(['v7_ST_CODE', 'v28_HHID'], as_index = False).agg({'v23_SPEND_RSS': 'sum'})
exp_food_U = exp_food_U.groupby(['v7_ST_CODE', 'v28_HHID'], as_index = False).agg({'v23_SPEND_RSS': 'sum'})

exp_food_R = exp_food_R.groupby(['v7_ST_CODE']).agg({'v23_SPEND_RSS': 'median'})
exp_food_U = exp_food_U.groupby(['v7_ST_CODE']).agg({'v23_SPEND_RSS': 'median'})

exp_food_R['HH_R_EXP_FOOD'] = exp_food_R['v23_SPEND_RSS']*0.013*12
exp_food_U['HH_U_EXP_FOOD'] = exp_food_U['v23_SPEND_RSS']*0.013*12

exp_food_R = exp_food_R.drop(['v23_SPEND_RSS'], axis = 1)
exp_food_U = exp_food_U.drop(['v23_SPEND_RSS'], axis = 1)

In [ ]:
#preparing HOUSING EXPENDITURE df
exp_housing_R = exp_housing_R.groupby(['v7_ST_CODE', 'v24_HHID'], as_index = False).agg({'v20_SPEND_RSS': 'sum'})
exp_housing_U = exp_housing_U.groupby(['v7_ST_CODE', 'v24_HHID'], as_index = False).agg({'v20_SPEND_RSS': 'sum'})

exp_housing_R = exp_housing_R.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})
exp_housing_U = exp_housing_U.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})

exp_housing_R['HH_R_EXP_HOUS'] = exp_housing_R['v20_SPEND_RSS']*0.013*12
exp_housing_U['HH_U_EXP_HOUS'] = exp_housing_U['v20_SPEND_RSS']*0.013*12

exp_housing_R = exp_housing_R.drop(['v20_SPEND_RSS'], axis = 1)
exp_housing_U = exp_housing_U.drop(['v20_SPEND_RSS'], axis = 1)

In [ ]:
#preparing TAXES EXPENDITURE df

exp_taxes_R = exp_taxes_R.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})
exp_taxes_U = exp_taxes_U.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})

exp_taxes_R['HH_R_EXP_TAX'] = exp_taxes_R['v20_SPEND_RSS']*0.013*12
exp_taxes_U['HH_U_EXP_TAX'] = exp_taxes_U['v20_SPEND_RSS']*0.013*12

exp_taxes_R = exp_taxes_R.drop(['v20_SPEND_RSS'], axis = 1)
exp_taxes_U = exp_taxes_U.drop(['v20_SPEND_RSS'], axis = 1)

In [ ]:
#preparing EXTRA EXPENDITURE df
exp_extra_R = exp_extra_R.groupby(['v7_ST_CODE', 'v24_HHID'], as_index = False).agg({'v20_SPEND_RSS': 'sum'})
exp_extra_U = exp_extra_U.groupby(['v7_ST_CODE', 'v24_HHID'], as_index = False).agg({'v20_SPEND_RSS': 'sum'})

exp_extra_R = exp_extra_R.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})
exp_extra_U = exp_extra_U.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})

exp_extra_R['HH_R_EXP_EXTRA'] = exp_extra_R['v20_SPEND_RSS']*0.013*12
exp_extra_U['HH_U_EXP_EXTRA'] = exp_extra_U['v20_SPEND_RSS']*0.013*12

exp_extra_R = exp_extra_R.drop(['v20_SPEND_RSS'], axis = 1)
exp_extra_U = exp_extra_U.drop(['v20_SPEND_RSS'], axis = 1)

In [ ]:
#preparing MOBILE EXPENDITURE df

exp_mobile_R = exp_mobile_R.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})
exp_mobile_U = exp_mobile_U.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})

exp_mobile_R['HH_R_EXP_MOB'] = exp_mobile_R['v20_SPEND_RSS']*0.013*12
exp_mobile_U['HH_U_EXP_MOB'] = exp_mobile_U['v20_SPEND_RSS']*0.013*12

exp_mobile_R = exp_mobile_R.drop(['v20_SPEND_RSS'], axis = 1)
exp_mobile_U = exp_mobile_U.drop(['v20_SPEND_RSS'], axis = 1)

In [ ]:
#preparing LANDLINE EXPENDITURE df

exp_landline_R = exp_landline_R.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})
exp_landline_U = exp_landline_U.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})

exp_landline_R['HH_R_EXP_LANDL'] = exp_landline_R['v20_SPEND_RSS']*0.013*12
exp_landline_U['HH_U_EXP_LANDL'] = exp_landline_U['v20_SPEND_RSS']*0.013*12

exp_landline_R = exp_landline_R.drop(['v20_SPEND_RSS'], axis = 1)
exp_landline_U = exp_landline_U.drop(['v20_SPEND_RSS'], axis = 1)

In [ ]:
#preparing INTERNET EXPENDITURE df

exp_internet_R = exp_internet_R.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})
exp_internet_U = exp_internet_U.groupby(['v7_ST_CODE']).agg({'v20_SPEND_RSS': 'median'})

exp_internet_R['HH_R_EXP_INTER'] = exp_internet_R['v20_SPEND_RSS']*0.013*12
exp_internet_U['HH_U_EXP_INTER'] = exp_internet_U['v20_SPEND_RSS']*0.013*12

exp_internet_R = exp_internet_R.drop(['v20_SPEND_RSS'], axis = 1)
exp_internet_U = exp_internet_U.drop(['v20_SPEND_RSS'], axis = 1)

In [ ]:
#merging together spending columns
spend_2011 = pd.concat([exp_R_total, exp_food_R, exp_housing_R, exp_taxes_R, exp_extra_R, exp_mobile_R, exp_landline_R, exp_internet_R, exp_U_total, exp_food_U, exp_housing_U, exp_taxes_U, exp_extra_U, exp_mobile_U, exp_landline_U, exp_internet_U], join='outer', axis=1)

In [ ]:
#resetting index
spend_2011.reset_index(inplace = True)

In [ ]:
#creating % columns
spend_2011['HH_R_EXP_FOOD_PC'] = spend_2011['HH_R_EXP_FOOD']/spend_2011['HH_R_EXP_T']
spend_2011['HH_R_EXP_HOUS_PC'] = spend_2011['HH_R_EXP_HOUS']/spend_2011['HH_R_EXP_T']
spend_2011['HH_R_EXP_TAX_PC'] = spend_2011['HH_R_EXP_TAX']/spend_2011['HH_R_EXP_T']
spend_2011['HH_R_EXP_EXTRA_PC'] = spend_2011['HH_R_EXP_EXTRA']/spend_2011['HH_R_EXP_T']
spend_2011['HH_R_EXP_MOB_PC'] = spend_2011['HH_R_EXP_MOB']/spend_2011['HH_R_EXP_T']
spend_2011['HH_R_EXP_LANDL_PC'] = spend_2011['HH_R_EXP_LANDL']/spend_2011['HH_R_EXP_T']
spend_2011['HH_R_EXP_INTER_PC'] = spend_2011['HH_R_EXP_INTER']/spend_2011['HH_R_EXP_T']

spend_2011['HH_U_EXP_FOOD_PC'] = spend_2011['HH_U_EXP_FOOD']/spend_2011['HH_U_EXP_T']
spend_2011['HH_U_EXP_HOUS_PC'] = spend_2011['HH_U_EXP_HOUS']/spend_2011['HH_U_EXP_T']
spend_2011['HH_U_EXP_TAX_PC'] = spend_2011['HH_U_EXP_TAX']/spend_2011['HH_U_EXP_T']
spend_2011['HH_U_EXP_EXTRA_PC'] = spend_2011['HH_U_EXP_EXTRA']/spend_2011['HH_U_EXP_T']
spend_2011['HH_U_EXP_MOB_PC'] = spend_2011['HH_U_EXP_MOB']/spend_2011['HH_U_EXP_T']
spend_2011['HH_U_EXP_LANDL_PC'] = spend_2011['HH_U_EXP_LANDL']/spend_2011['HH_U_EXP_T']
spend_2011['HH_U_EXP_INTER_PC'] = spend_2011['HH_U_EXP_INTER']/spend_2011['HH_U_EXP_T']

## Part 2b: Retail ISP (B2C) - Resulting Tables
<br>
Putting together excel table for Morgan

In [ ]:
#merging everything for resulting tables
Result = pd.merge(IND_Subdistrics_B2C_G, R_U_G, on = 'D_CODE', how = 'outer')
Result = pd.merge(Result, MDU_T, left_on = 'D_CODE', right_on = 'CODE', how ='outer')
Result['ST_CODE'] = Result['D_CODE'].str[:2]
Result = pd.merge(Result, spend_2011, left_on = 'ST_CODE', right_on = 'v7_ST_CODE', how ='outer')
Result['MDU_T'] = Result['MDU_PC']*Result['HH_T']

In [ ]:
#dropping useliss columns
Result = Result.drop(['AVG_MBPS', 'MAX_MED_MBPS', 'latency', 'Inhabited Villages', 'Uninhabited Villages', 'Number of towns', 'CODE', 'v7_ST_CODE'], axis =1)

In [ ]:
#renaming according to agreed convention
Result.rename(columns={'2Mbps_PC':'F_2Mbps_PC',\
                       '5Mbps_PC': 'F_5Mbps_PC',\
                       '10Mbps_PC': 'F_10Mbps_PC',\
                       '15Mbps_PC': 'F_15Mbps_PC',\
                       '20Mbps_PC': 'F_20Mbps_PC',\
                       '25Mbps_PC': 'F_25Mbps_PC',\
                       '50Mbps_PC': 'F_50Mbps_PC',\
                       '100Mbps_PC': 'F_100Mbps_PC',\
                       '400Mbps_PC': 'F_400Mbps_PC',\
                       'area_Vi_cover_PC': 'Vi_cover_PC',\
                       'area_MTNL_cover_PC': 'MTNL_cover_PC',\
                       'area_BSNL_cover_PC': 'BSNL_cover_PC',\
                       'area_Airtel_cover_PC': 'Airtel_cover_PC'},inplace=True)


In [ ]:
#ensuring correct order
lst = ['D_CODE',\
                 'Population',\
                 'HH_T',\
                 'Area',\
                 'PD',\
                 'MDU_T',\
                 'MDU_PC',\
                 'HH_INC_Q10_PC',\
                 'HH_INC_Q20_PC',\
                 'HH_INC_Q30_PC',\
                 'HH_INC_Q40_PC',\
                 'HH_INC_Q50_PC',\
                 'HH_INC_Q60_PC',\
                 'HH_INC_Q70_PC',\
                 'HH_INC_Q80_PC',\
                 'HH_INC_Q90_PC',\
                 'HH_INC_Q100_PC',\
                 'HH_R_EXP_T',\
                 'HH_R_EXP_FOOD',\
                 'HH_R_EXP_HOUS',\
                 'HH_R_EXP_TAX',\
                 'HH_R_EXP_EXTRA',\
                 'HH_R_EXP_MOB',\
                 'HH_R_EXP_LANDL',\
                 'HH_R_EXP_INTER',\
                 'HH_U_EXP_T',\
                 'HH_U_EXP_FOOD',\
                 'HH_U_EXP_HOUS',\
                 'HH_U_EXP_TAX',\
                 'HH_U_EXP_EXTRA',\
                 'HH_U_EXP_MOB',\
                 'HH_U_EXP_LANDL',\
                 'HH_U_EXP_INTER',\
                 'HH_R_EXP_FOOD_PC',\
                 'HH_R_EXP_HOUS_PC',\
                 'HH_R_EXP_TAX_PC',\
                 'HH_R_EXP_EXTRA_PC',\
                 'HH_R_EXP_MOB_PC',\
                 'HH_R_EXP_LANDL_PC',\
                 'HH_R_EXP_INTER_PC',\
                 'HH_U_EXP_FOOD_PC',\
                 'HH_U_EXP_HOUS_PC',\
                 'HH_U_EXP_TAX_PC',\
                 'HH_U_EXP_EXTRA_PC',\
                 'HH_U_EXP_MOB_PC',\
                 'HH_U_EXP_LANDL_PC',\
                 'HH_U_EXP_INTER_PC']

lst.extend(afford)
lst.extend(['MED_MBPS',\
                 'number_speed_tests',\
                 'F_2Mbps_PC',\
                 'F_5Mbps_PC',\
                 'F_10Mbps_PC',\
                 'F_15Mbps_PC',\
                 'F_20Mbps_PC',\
                 'F_25Mbps_PC',\
                 'F_50Mbps_PC',\
                 'F_100Mbps_PC',\
                 'F_400Mbps_PC',\
                 'Vi_cover_PC',\
                 'MTNL_cover_PC',\
                 'BSNL_cover_PC',\
                 'Airtel_cover_PC'])
Result = Result[lst]

In [ ]:
#Result.to_sql(name='IND_B2C', con=engine, if_exists = 'replace', chunksize = 10000, index=False)

In [ ]:
#Result = Result.to_excel('India Output.xlsx')

# Appendix: Income level estimation
The following income level estimation resulting in fitted distributions for rural and urban RWI attributes in India. This analysis is persormed only ones and once distributions are found, code is kept just for reference.

In [ ]:
#Distribution = pd.read_excel('Distribution data.xlsx')

In [ ]:
#Distribution = Distribution.drop(['ID' ,'X', 'Y'], axis = 1)
#Distribution = Distribution.rename(columns={"RWIRaster": "RWI", "indpd20201k": "POP"})

In [ ]:
#Distribution_rural_5k = Distribution[Distribution['POP']<=5000]
#Distribution_urban_5k = Distribution[Distribution['POP']>5000]

In [ ]:
#Distribution_rural_5k['POP'] = Distribution_rural_5k['POP'].round()
#Distribution_urban_5k['POP'] = Distribution_urban_5k['POP'].round()

In [ ]:
#Distribution_rural_5k = Distribution_rural_5k.dropna()
#Distribution_urban_5k = Distribution_urban_5k.dropna()

In [ ]:
'''rural_income_observation = []

for index, row in Distribution_rural_5k.iterrows():
    print(index)
    count = row['POP']
    while count > 0:
        rural_income_observation.append(row['RWI'])
        count = count-1'''

In [ ]:
'''urban_income_observation = []

for index, row in Distribution_urban_5k.iterrows():
    print(index)
    count = row['POP']
    while count > 0:
        urban_income_observation.append(row['RWI'])
        count = count-1'''

In [ ]:
#rural_income_observation_pd = pd.DataFrame(rural_income_observation)

In [ ]:
#urban_income_observation_pd = pd.DataFrame(urban_income_observation)

In [ ]:
#rural_income_observation_pd.to_csv('rural_income_observations.csv')

In [ ]:
#urban_income_observation_pd.to_csv('urban_income_observations.csv')

In [ ]:
'''import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

def full_fit(data, distribution, timeout):

    f = Fitter(data, distributions = distribution, timeout = None)
    f.fit()
    return f.summary()'''

In [ ]:
#urban_sample_1 = urban_income_observation_pd.sample(n = 250000)
#urban_sample_2 = urban_income_observation_pd.sample(n = 250000)
#urban_sample_3 = urban_income_observation_pd.sample(n = 250000)

In [ ]:
#rural_sample_1 = rural_income_observation_pd.sample(n = 250000)
#rural_sample_2 = rural_income_observation_pd.sample(n = 250000)
#rural_sample_3 = rural_income_observation_pd.sample(n = 250000)

In [ ]:
'''sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.displot(data=rural_sample_1, kind="hist", bins = 100, aspect = 1.5)'''

In [ ]:
'''sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.displot(data=rural_sample_2, kind="hist", bins = 100, aspect = 1.5)'''

In [ ]:
'''sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.displot(data=rural_sample_3, kind="hist", bins = 100, aspect = 1.5)'''

In [ ]:
'''sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.displot(data=urban_sample_1, kind="hist", bins = 100, aspect = 1.5)'''

In [ ]:
'''sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.displot(data=urban_sample_2, kind="hist", bins = 100, aspect = 1.5)'''

In [ ]:
'''sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.displot(data=urban_sample_3, kind="hist", bins = 100, aspect = 1.5)'''

In [ ]:
#rural_RWI_1 = rural_sample_1.values
#rural_RWI_2 = rural_sample_2.values
#rural_RWI_3 = rural_sample_3.values

#urban_RWI_1 = urban_sample_1.values
#urban_RWI_2 = urban_sample_2.values
#urban_RWI_3 = urban_sample_3.values

In [ ]:
#get_distributions()

In [ ]:
'''result = pd.DataFrame(columns = ['sumsquare_error', 'aic', 'bic', 'kl_div'])

for distribution in log_progress(['t',\
                                  'trapezoid', 'trapz', 'triang', 'truncexpon', 'truncnorm', 'tukeylambda', 'uniform',\
                                  'vonmises', 'vonmises_line', 'wald', 'weibull_max', 'weibull_min', 'wrapcauchy']):
    distribution = full_fit(urban_RWI_1, distribution, None)
    result = result.append(distribution, ignore_index = True)
    print(distribution)'''

In [ ]:
'''f = Fitter(rural_RWI_1, distributions = ['gausshyper'], timeout = None)
f.fit()
f.summary()
f.get_best(method = 'sumsquare_error')'''

In [ ]:
'''f = Fitter(rural_RWI_2, distributions = ['powernorm'], timeout = None)
f.fit()
f.summary()
f.get_best(method = 'sumsquare_error')'''

In [ ]:
'''f = Fitter(rural_RWI_3, distributions = ['powernorm'], timeout = None)
f.fit()
f.summary()
f.get_best(method = 'sumsquare_error')'''